<a href="https://colab.research.google.com/github/AhmedNusayer/Book-Self/blob/master/Data_Collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import os
import tensorflow as tf
import six.moves.urllib as urllib
import sys
import tarfile
import zipfile
import glob as glob


from matplotlib import pyplot as plt
from PIL import Image
from collections import defaultdict
from io import StringIO
from IPython.display import display

!pip install tf_slim
!pip install pycocotools

     |████████████████████████████████| 352 kB 5.5 MB/s 


In [ ]:
%%bash
cd drive/MyDrive/Thesis/models/research/
protoc object_detection/protos/*.proto --python_out=.

In [ ]:
%%bash 
cd drive/MyDrive/Thesis/models/research
pip install .

Processing /content/drive/MyDrive/Thesis/models/research
  Created wheel for slim: filename=slim-0.1-py3-none-any.whl size=2091894 sha256=a710ea51c2103d9094c5ef22bc7a8db7e7681ace8bda05235ff221fd66f7eee3
  Stored in directory: /tmp/pip-ephem-wheel-cache-u3s3tqit/wheels/ad/4a/b7/776def82097187bbbafa72970277967a625fa43a70836004a6
Successfully built slim


  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.


In [ ]:
%matplotlib inline
import sys
# Append your Tensorflow object detection and darkflow directories to your path
sys.path.append('PATH_TO_TENSORFLOW_OBJECT_DETECTION_FOLDER') # ~/tensorflow/models/research/object_detection
sys.path.append('PATH_TO_DARKFLOW_FOLDER') # ~/darkflow

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [ ]:
EPSILON = 1e-5
# Load GTSDB validation images
CLASSES = ['prohibitory','mandatory','danger']
#Path to the German Traffic Sign Detection Benchmark dataset
DATASET_PATH = '~/Datasets/GermanTSDataset/Detection'
full_set = ['%05d.jpg' % x for x in range(900)]
valid_set = full_set[600:900]

In [ ]:
category_index = {1: {'id': 1, 'name': 'prohibitory'},
 2: {'id': 2, 'name': 'mandatory'},
 3: {'id': 3, 'name': 'danger'}}
categories = [{'id': 1, 'name': 'prohibitory'},
 {'id': 2, 'name': 'mandatory'},
 {'id': 3, 'name': 'danger'}]

In [ ]:
MODEL_NAME = 'faster_rcnn_inception_resnet_v2_atrous'
# MODEL_NAME = 'faster_rcnn_resnet_101'
# MODEL_NAME = 'faster_rcnn_resnet50'
# MODEL_NAME = 'faster_rcnn_inception_v2'
# MODEL_NAME = 'rfcn_resnet101'
# MODEL_NAME = 'ssd_inception_v2'
# MODEL_NAME = 'ssd_mobilenet_v1'
# MODEL_NAME = 'yolo_v2'

In [ ]:
# Path to frozen detection graph. This is the actual model that is used for the traffic sign detection.
#MODEL_PATH = os.path.join('models', MODEL_NAME)
#PATH_TO_CKPT = os.path.join(MODEL_PATH,'inference_graph/frozen_inference_graph.pb')
PATH_TO_CKPT = 'drive/MyDrive/Thesis/models/faster_rcnn_inception_resnet_v2_atrous/inference_graph/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('gtsdb_data', 'gtsdb_label_map.pbtxt')

NUM_CLASSES = 3

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [ ]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)
print(label_map)

item {
  name: "prohibitory"
  id: 1
}
item {
  name: "mandatory"
  id: 2
}
item {
  name: "danger"
  id: 3
}



In [ ]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    print(im_width)
    print(im_height)
    print(image.getdata())
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

In [ ]:
import cv2

PATH_TO_TEST_IMAGES_DIR = 'TEST_IMAGE_PATHS'
# Image directory
os.chdir('/content/TEST_IMAGE_PATHS')
print(os.getcwd())

for root, dirs, files in os.walk(os.getcwd()):
    for name in files:
        image = cv2.imread(name)
        if name.endswith((".png", ".PNG", ".JPG")):
          fileName = os.path.splitext(name)[0]
          save_as = fileName + ".jpg"
          cv2.imwrite(save_as, image, [int(cv2.IMWRITE_JPEG_QUALITY), 100])
          os.remove(name)

os.chdir('/content')
print(os.getcwd())

TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, '*jpg'))
print(TEST_IMAGE_PATHS)
#TEST_IMAGE_PATHS = "TEST_IMAGE_PATHS/"

# Size, in inches, of the output images.
IMAGE_SIZE = (20, 20)

/content/TEST_IMAGE_PATHS
/content
['TEST_IMAGE_PATHS/V0046_1830.jpg', 'TEST_IMAGE_PATHS/V0046_4320.jpg']




Frame Extractor From Video




In [ ]:
import os
import numpy as np
import cv2
from glob import glob

def save_frame(video_path, save_dir, gap=10):
    vid_name = os.path.basename(video_path)
    name = os.path.splitext(vid_name)[0]
    cap = cv2.VideoCapture(video_path)
    idx = 0

    while True:
        ret, frame = cap.read()
        if ret == False:
            cap.release()
            break

        if idx == 0 or idx % gap == 0:
            cv2.imwrite("drive/MyDrive/Thesis/VideoFrames/" + name + "_" + str(idx) + ".jpg", frame)
        idx += 1

if __name__ == "__main__":
    video_paths = glob("/content/drive/MyDrive/Thesis/Videos/*")
    save_dir = "/content/TEST_IMAGE_PATHS"

    for path in video_paths:
        save_frame(path, save_dir, gap=10)

In [ ]:
PATH_TO_TEST_IMAGES_DIR = 'drive/MyDrive/Thesis/VideoFrames'
TEST_IMAGE_PATHS = glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, '*jpg'))
print(TEST_IMAGE_PATHS)

['drive/MyDrive/Thesis/VideoFrames/V0049_0.jpg', 'drive/MyDrive/Thesis/VideoFrames/V0049_10.jpg', 'drive/MyDrive/Thesis/VideoFrames/V0049_20.jpg', 'drive/MyDrive/Thesis/VideoFrames/V0049_30.jpg', 'drive/MyDrive/Thesis/VideoFrames/V0049_40.jpg', 'drive/MyDrive/Thesis/VideoFrames/V0049_50.jpg', 'drive/MyDrive/Thesis/VideoFrames/V0049_60.jpg', 'drive/MyDrive/Thesis/VideoFrames/V0049_70.jpg', 'drive/MyDrive/Thesis/VideoFrames/V0049_80.jpg', 'drive/MyDrive/Thesis/VideoFrames/V0049_90.jpg', 'drive/MyDrive/Thesis/VideoFrames/V0049_100.jpg', 'drive/MyDrive/Thesis/VideoFrames/V0049_110.jpg', 'drive/MyDrive/Thesis/VideoFrames/V0049_120.jpg', 'drive/MyDrive/Thesis/VideoFrames/V0049_130.jpg', 'drive/MyDrive/Thesis/VideoFrames/V0049_140.jpg', 'drive/MyDrive/Thesis/VideoFrames/V0049_150.jpg', 'drive/MyDrive/Thesis/VideoFrames/V0049_160.jpg', 'drive/MyDrive/Thesis/VideoFrames/V0049_170.jpg', 'drive/MyDrive/Thesis/VideoFrames/V0049_180.jpg', 'drive/MyDrive/Thesis/VideoFrames/V0049_190.jpg', 'drive/MyD

In [ ]:
with detection_graph.as_default():
    with tf.compat.v1.Session(graph=detection_graph) as sess:
        for idx, image_path in enumerate(TEST_IMAGE_PATHS):
            image = Image.open(image_path)
            path = image_path.split("drive/MyDrive/Thesis/VideoFrames/",1)[1]
            print(path)
            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            image_np = load_image_into_numpy_array(image)
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            # Each box represents a part of the image where a particular object was detected.
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            
            # Each score represent how level of confidence for each of the objects.
            # Score is shown on the result image, together with the class label.
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            # Actual detection.
            (boxes, scores, classes, num_detections) = sess.run(
                [boxes, scores, classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            #print(num_detections)            
            
            final_score = np.squeeze(scores)    
            count = 0
            for i in range(100):
                if scores is None or final_score[i] > 0.5:
                    count = count + 1
                    cv2.imwrite('drive/MyDrive/Thesis/ExtractedImages/' + path , cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR))
                    break
            print(count)


V0049_0.jpg
1280
720
0
V0049_10.jpg
1280
720
0
V0049_20.jpg
1280
720
0
V0049_30.jpg
1280
720
0
V0049_40.jpg
1280
720
0
V0049_50.jpg
1280
720
0
V0049_60.jpg
1280
720
0
V0049_70.jpg
1280
720
0
V0049_80.jpg
1280
720
0
V0049_90.jpg
1280
720
0
V0049_100.jpg
1280
720
1
V0049_110.jpg
1280
720
1
V0049_120.jpg
1280
720
1
V0049_130.jpg
1280
720
0
V0049_140.jpg
1280
720
0
V0049_150.jpg
1280
720
0
V0049_160.jpg
1280
720
0
V0049_170.jpg
1280
720
0
V0049_180.jpg
1280
720
0
V0049_190.jpg
1280
720
0
V0049_200.jpg
1280
720
0
V0049_210.jpg
1280
720
0
V0049_220.jpg
1280
720
0
V0049_230.jpg
1280
720
0
V0049_240.jpg
1280
720
0
V0049_250.jpg
1280
720
0
V0049_260.jpg
1280
720
0
V0049_270.jpg
1280
720
0
V0049_280.jpg
1280
720
0
V0049_290.jpg
1280
720
0
V0049_300.jpg
1280
720
0
V0049_310.jpg
1280
720
0
V0049_320.jpg
1280
720
0
V0049_330.jpg
1280
720
0
V0049_340.jpg
1280
720
0
V0049_350.jpg
1280
720
0
V0049_360.jpg
1280
720
0
V0049_370.jpg
1280
720
0
V0049_380.jpg
1280
720
0
V0049_390.jpg
1280
720
0
V0049_400.j

KeyboardInterrupt: ignored